# <center><span style="font-size: 42px;color: darkgreen;">Projeto - Importando Dados do Banco de Dados Oracle Para o HDFS</center></span>

<br>

---

<br>

# Contexto

<br>

A **Oracle** é líder mundial em banco de dados e uma das gigantes da tecnologia. Seus bancos de dados são amplamente utilizados em Data Warehouses ou sistemas ERP, como **SAP**, **PeopleSoft** e **JD Edwards**. Esses sistemas frequentemente armazenam milhões de registros, tornando essencial o uso de soluções eficientes para manipulação e análise desses dados.

Neste projeto, apresentaremos, passo a passo, como importar dados do banco de dados Oracle para o HDFS utilizando o **Sqoop**, uma ferramenta **ETL gratuita** e parte integrante do ecossistema Hadoop. O **Sqoop** permite a conexão via **JDBC** ao banco de dados Oracle, a execução de queries, a extração de dados e o carregamento no **HDFS**, possibilitando posterior processamento analítico em um cluster.

O projeto será realizado em uma **Máquina Virtual (VM)**, configurada no **Oracle VirtualBox**, com todos os componentes necessários, incluindo Hadoop, Sqoop e banco de dados Oracle.  


<br><br>

---

<br>

# Problema de Negócio

<br>

> Como mover dados de um banco de dados Oracle para o HDFS de maneira eficiente, utilizando ferramentas do ecossistema Hadoop?

<br><br>

---

<br>

# Sobre o Dataset

<br>

Para este projeto, utilizaremos um dataset contendo informações de empresas globais. O arquivo, chamado **`companies_sorted.csv`**, possui milhões de registros e um tamanho total de aproximadamente 1GB. O objetivo é explorar as ferramentas para mover os dados do banco de dados **Oracle**, onde serão carregados, para o **HDFS**.

<br>

### **Descrição do Dataset**

O dataset contém as seguintes colunas:

| **Coluna**                 | **Descrição**                                                             | **Tipo**     |
|----------------------------|---------------------------------------------------------------------------|--------------|
| `Unnamed: 0`               | Identificador único da entrada.                                           | int64        |
| `name`                     | Nome da empresa.                                                         | object       |
| `domain`                   | Domínio ou URL da empresa.                                               | object       |
| `year founded`             | Ano de fundação da empresa.                                              | float64      |
| `industry`                 | Setor da indústria a que a empresa pertence.                             | object       |
| `size range`               | Faixa de tamanho da empresa (ex.: 10001+).                               | object       |
| `locality`                 | Localidade, incluindo cidade e estado.                                   | object       |
| `country`                  | País de operação.                                                        | object       |
| `linkedin url`             | URL do perfil da empresa no LinkedIn.                                    | object       |
| `current employee estimate`| Estimativa atual de funcionários da empresa.                             | int64        |
| `total employee estimate`  | Estimativa total de funcionários ao longo do tempo.                      | int64        |


<br><br>

---

<br>


## Alternando Entre Usuários

- **Conectar como usuário root**: `su` (digitar senha)
- **Digitar**: `su - nome_usuario`

<br><br>

---

<br>


# <center><u><span style="font-size: 34px;color: darkgreen;">Início do Projeto</span></center></u>

<br>


# 1. Iniciando os Serviços

<br>

Ná **máquina virtual** executar os comandos abaixo:

<br>

- **1.1 Iniciar o Listener digitando no terminal**:
   ```java
   lsnrctl start  |  lsnrctl stop
   ```
- **1.2 Iniciar o Banco de Dados digitando no terminal**:
   ```java
   sqlplus / as sysdba
   ```
- **1.3 Dentro do terminal do Oracle, digitar**:
   ```java
   startup  |  shutdown immediate
   ```
- **1.4 Verificar Status**:
   ```java
   lsnrctl status
   ```
   
<br><br><br>

# 2. Carregando Dados Para o Banco Oracle

<br>

### 2.1 Criando Um Schema (se desejar criar um Schema novo ou usar *curso*)

<br>

A seção **2.1 Criando Um Schema** é **opcional** e deve ser seguida apenas se você desejar criar um **novo schema**. Caso o schema `curso` já exista no banco de dados, você pode pular diretamente para a **seção 2.2 Conectando com o Schema `curso`**, onde é detalhado como se conectar e utilizar o schema existente.

<br>

- **2.1.1 Acessar Terminal do sqlplus**:
   ```java
   sqlplus / as sysdba
   ```
- **2.1.2  Criando o schema (usuário) `curso`**:
   ```java
   create user curso identified by digitar_senha;
   ```
- **2.1.3 Conceder previlégios necessários para `curso`**:
   ```java
   grant connect, resource, unlimited tablespace to curso;
   ```
- **2.1.4 Sair do terminal do sqlplus**:
   ```java
   exit
   ```
   
<br><br>

### 2.2 Conectando com o Schema `curso`

<br>

- **2.2.1 Acessar Terminal do sqlplus como `curso` (digitar senha)**:
   ```bash
   sqlplus curso@orcl
   ```
   
<br><br>

### 2.3 Criando uma Tabela

<br>

- **2.3.1 Acessar Terminal do sqlplus como `curso` e digitar**:
   ```sql
CREATE TABLE companies (ID NUMBER PRIMARY KEY, NAME VARCHAR2(500), DOMAIN VARCHAR2(255), YEAR_FOUNDED VARCHAR2(30), INDUSTRY VARCHAR2(255), SIZE_RANGE VARCHAR2(50), LOCALITY VARCHAR2(255), COUNTRY VARCHAR2(100), LINKEDIN_URL VARCHAR2(800), CURRENT_EMPLOYEE_ESTIMATE VARCHAR2(100), TOTAL_EMPLOYEE_ESTIMATE VARCHAR2(255));

   SELECT table_name FROM user_tables;
   ```
   
<br><br>

### 2.4 Preparando a Carga de Dados

<br>

#### 2.4.1 Realizando `Limpeza`nos dados antes de enviar para o banco oracle e `salvando`:

<br>

```python
# Carregar o arquivo em duas partes
import pandas as pd

filename = "Dataset/combined_companies_dataset.csv"

# Verificar o número de linhas no arquivo
num_lines = sum(1 for line in open(filename))
print(f"O arquivo tem {num_lines} linhas.")

# Carregar a primeira metade
first_half = pd.read_csv(filename, nrows=num_lines // 2)
print(f"\nPrimeira parte carregada com {len(first_half)} linhas.\n\n\n")

# Carregar a segunda metade
second_half = pd.read_csv(
    filename,
    skiprows=(num_lines // 2 + 1),  # Ignorar o cabeçalho extra
    names=first_half.columns,  # Reutilizar os nomes das colunas da primeira parte
    header=None  # Indicar que não há cabeçalho no que será carregado
)
print(f"\nSegunda parte carregada com {len(second_half)} linhas.")

# Converter para o tipo Int64, que permite valores NaN
first_half['year_founded'] = first_half['year_founded'].astype('Int64')
second_half['year_founded'] = second_half['year_founded'].astype('Int64')

# Remover aspas apenas nas colunas de texto
text_columns = first_half.select_dtypes(include=['object']).columns
first_half[text_columns] = first_half[text_columns].applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)
text_columns = second_half.select_dtypes(include=['object']).columns
second_half[text_columns] = second_half[text_columns].applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)

# Substituir vírgulas por espaço e hífen apenas nas colunas de texto
text_columns = first_half.select_dtypes(include=['object']).columns
first_half[text_columns] = first_half[text_columns].applymap(lambda x: x.replace(',', ' -') if isinstance(x, str) else x)
text_columns = second_half.select_dtypes(include=['object']).columns
second_half[text_columns] = second_half[text_columns].applymap(lambda x: x.replace(',', ' -') if isinstance(x, str) else x)

# Excluir a primeira coluna
first_half = first_half.drop(columns=first_half.columns[0])
second_half = second_half.drop(columns=second_half.columns[0])

# Salvar o arquivo com vírgula como separador
first_half.to_csv("Dataset/campanies_first_half_no_header.csv", index=False, header=False, sep=",")
print('\nArquivo first_half salvo.')

second_half.to_csv("Dataset/campanies_second_half_no_header.csv", index=False, header=False, sep=",")
print('\nArquivo second_half salvo.')
```

<br><br>

Para carregar dados no **Oracle**, usamos o **SQL*Loader**. Este aplicativo requer um **control file**:

<br>

- **2.4.2 Navegar até diretório *home* criar um novo diretório chamado `etl`**:
   ```bash
   mkdir etl
   ```
- **2.4.3 Dentro do diretório `etl` criar o arquivo `loader.dat` digitando no terminal**:
   ```bash
   gedit loader.dat
   ```
- **2.4.4 No arquivo `loader.dat` colar o conteúdo e salvar**:
   ```data
load data
INFILE '/home/oracle/Documents/Datasets/campanies_first_half_no_header.csv'
INTO TABLE companies
APPEND
FIELDS TERMINATED BY ','
trailing nullcols
(id SEQUENCE (MAX,1),
name CHAR(255),
domain CHAR(255),
year_founded CHAR(6),
industry CHAR(255),
size_range CHAR(50),
locality CHAR(255),
country CHAR(100),
linkedin_url CHAR(255),
current_employee_estimate CHAR(20),
total_employee_estimate CHAR(20))
   ```
   
- **2.4.5: Executando o arquivo `loader.dat` no terminal do diretório `etl`**:
   ```bash
   sqlldr userid=curso/digitar_senha control=loader.dat log=loader.log
   ```
- **2.4.6 Verificar Arquivo de Log**:
   ```bash
   gedit loader.log
   ```
   
<br><br>


### 2.5 Verificando Dados Carregados no Banco Oracle

<br>

- **2.5.1 Acessar Terminal do sqlplus como `curso` (digitar senha) ou outro schema/usuário**:
   ```bash
   sqlplus curso@orcl
   ```
- **2.5.2 Verificando tabelas no schema `curso`**:
   ```bash
   select table_name FROM user_tables;
   ```
- **2.5.3 Visualizando *count* (*quantidade*) de linhas da tabela `cinema`**:
   ```bash
   select count(*) from companies;
   ```
- **2.5.4 Visualizando as 5 primeiras linhas da tabela `cinema`**:
   ```bash
   SELECT * FROM companies WHERE ROWNUM <= 5;
   ```   
- **2.5.5 Visualizando 5 linhas aleatórias da tabela `cinema`**:
   ```bash
   SELECT * FROM companies ORDER BY dbms_random.value FETCH FIRST 5 ROWS ONLY;
   ```
- **2.5.6 Ajustando a exibição temporária para melhor visualizar**:
   ```bash
   SET LINESIZE 100
   SET PAGESIZE 20
   ```  

<br><br><br>


# 3. Configurando o Sistema Para Importação de Dados do Banco de Dados Oracle para o HDFS com o Apache Sqoop

<br>

## 3.1 Iniciando os Serviços

<br>

####  <u>Hadoop</u>

> **<u>Importante</u>**: é necessário está logado com o usuário **hadoop** para inicializar os **serviços do hadoop**.

<br>

- **3.1.1 Iniciar o HDFS (NameNode, DataNode, SecondaryNameNode)**:
   ```bash
   start-dfs.sh  |  stop-dfs.sh
   ```
- **3.1.2 Iniciar o YARN (ResourceManager, NodeManager)**:
   ```bash
   start-yarn.sh  |  stop-yarn.sh
   ```
- **3.1.3 Verificando serviços**:
   ```bash
   jps
   ```

<br>

#### <u>Oracle</u>

> **<u>Importante</u>**: é necessário está logado com o usuário **oracle** para inicializar o **banco oracle**.

<br>

- **3.1.4 Iniciar o Listener digitando no terminal**:
   ```java
   lsnrctl start  |  lsnrctl stop
   ```
- **3.1.5 Iniciar o Banco de Dados digitando no terminal**:
   ```java
   sqlplus / as sysdba
   ```
- **3.1.6 Dentro do terminal do Oracle, digitar**:
   ```java
   startup  |  shutdown immediate  |  exit
   ```
- **3.1.7 Verificar Status**:
   ```java
   lsnrctl status
   ```
- **3.1.8 Acessar Terminal do sqlplus como `curso` (digitar senha) ou outro schema/usuário**:
   ```bash
   sqlplus curso@orcl
   ```
- **3.1.9 Verificando tabelas no schema `curso`** (iremos usar a tabela **companies_hadoop** que foi clonada a partir da tabela **companies**):
   ```bash
   select table_name FROM user_tables;
   ```

<br><br>

## 3.2 Configurando o Driver JDBC

<br>

O `Apache Sqoop` conecta-se ao `Banco Oracle` usando o **driver JDBC**.

Cada **banco relacional** possui seu próprio **driver JDBC**, como o `MySQL` e o `PostgreSQL`. 

No caso do` Oracle`, é necessário fazer o **download do driver JDBC** específico diretamente no site da Oracle:

https://www.oracle.com/database/technologies/jdbc-ucp-122-downloads.html

Após o download, acesse a pasta onde o arquivo foi descompactado, localize o `ojdbc8.jar` e mova-o para o diretório do `Sqoop`, permitindo que o `Sqoop` reconheça o **driver JDBC**. O procedimento para drivers de outros bancos é o mesmo. No terminal digite:

- Conectar como usuário root: `su` (depois digite a senha)
- Como usuário root copie o arquivo com: `cp ojdbc8.jar /opt/sqoop/lib`
- Acessar diretório do sqoop: `cd /opt/sqoop/lib/`
- Alterar propriedade do arquivo ojdbc8.jar: `chown hadoop:hadoop ojdbc8.jar`
- Alterar a propriedade do arquivo commons-lang-2.6.jar: `chown hadoop:hadoop commons-lang-2.6.jar`
- Sair do usuário root: `exit`

<br><br>

## 3.3 Ajustando os Previlégios de Acesso

<br>

Em um **Ambiente de Produção** em empresas de médio e grande porte, cada *software* normalmente é executado em um servidor separado. Por exemplo:

- Um servidor é dedicado ao `Banco Oracle`,
- Outro servidor é dedicado ao `Apache Sqoop`,
- E uma máquina ou um grupo de máquinas são dedicadas ao `Apache HDFS`.

Neste laboratório, configuramos um **Ambiente de Teste** em uma única máquina. Nessa configuração, todos os serviços estão instalados, mas distribuídos em **diferentes usuários** no mesmo sistema operacional. Essa configuração gera desafios, pois cada serviço possui seu próprio usuário.

Dentro deste cenário, o **usuário oracle**, que executa o `Banco Oracle`, atualmente não possui acesso ao `HDFS`, pois ele está instalado no **usuário hadoop**. Para que o `Sqoop` possa acessar o `HDFS` e funcionar corretamente, é necessário **conceder acesso ao usuário oracle** para que ele consiga interagir com o `HDFS`.

O **objetivo** é **ajustar os privilégios de acesso** de modo que os diferentes usuários no sistema possam interagir com os serviços necessários, permitindo o uso completo de todos os serviços em uma única máquina de teste.

<br>

- **3.3.1 No `usuario oracle`, configurar as variáveis de ambiente para o Hadoop e Sqoop**.
  - Ir ao terminal e digitar: `gedit .bashrc`
  - No arquivo .bashrch colar o conteúdo:
   ```code
    # Java JDK
    export JAVA_HOME=/opt/jdk
    export PATH=$PATH:$JAVA_HOME/bin
    
    # Hadoop
    export HADOOP_HOME=/opt/hadoop
    export HADOOP_INSTALL=$HADOOP_HOME
    export HADOOP_COMMON_HOME=$HADOOP_HOME
    export HADOOP_MAPRED_HOME=$HADOOP_HOME
    export HADOOP_HDFS_HOME=$HADOOP_HOME
    export YARN_HOME=$HADOOP_HOME
    export PATH=$PATH:$HADOOP_HOME/bin:$HADOOP_HOME/sbin
    
    # Sqoop
    export SQOOP_HOME=/opt/sqoop
    export PATH=$PATH:$SQOOP_HOME/bin
    export HCAT_HOME=/opt/sqoop/hcatalog
    export ACCUMULO_HOME=/opt/sqoop/accumulo
   ```
   
   - Após salvar digite: `source .bashrc`

<br>

- **3.3.2 No `usuario hadoop`, definir os privilégios com os comandos abaixos**: 

```code
    hdfs dfsadmin -safemode leave
    hdfs dfs -chmod -R 777 /
    chmod -R 777 /opt/hadoop/logs
    exit
```
<br>

- **3.3.3 No `usuario root`, definir os privilégios com os comandos abaixos**:

```code
    groups oracle
    usermod -a -G hadoop oracle
    groups oracle
    exit
```

<br>

- **3.3.4 No `usuario oracle`, testar digitando no terminal**:

```code
    hdfs dfs -ls /
```

<br><br><br>

# 4. Transferindo os Dados do Oracle para o HDFS com o Apache Sqoop

<br>

- **4.1 Não esquecer de logar como usuário `hadoop` e após iniciar o serviços digitar no terminal**:

```code
hdfs dfsadmin -safemode leave
```

<br>

- **4.2 No `usuario oracle`, acessar o diretório `etl` e digitar no terminal**:

```code
sqoop import --connect jdbc:oracle:thin:curso/0804@dataserver.localdomain:1539/orcl --username curso --password 0804 --query "SELECT NAME, DOMAIN, YEAR_FOUNDED, INDUSTRY, SIZE_RANGE, LOCALITY, COUNTRY, LINKEDIN_URL, CURRENT_EMPLOYEE_ESTIMATE, TOTAL_EMPLOYEE_ESTIMATE FROM COMPANIES_HADOOP WHERE \$CONDITIONS" --target-dir /user/oracle/output --delete-target-dir -m 1
```

<br>

- **4.3 Checar se diretório `oracle` foi criado com**:

```code
hdfs dfs -ls /user
hdfs dfs -ls /user/oracle
hdfs dfs -ls /user/oracle/output
```

<br>

- **4.4 Visualizando conteúdo**:

```code
hdfs dfs -cat /user/oracle/output/part-m-00000
```

<br>

- **4.5 Verificando quantidade de linhas**:

```code
hdfs dfs -cat /user/oracle/output/part-m-00000 | wc -l
```

<br>

- **4.6 No diretório `etl` foi gerado o arquivo `QueryResult.java` que contém o código do MapReduce necessário para importação dos dados**:

```code
gedit QueryResult.java
```

<br>

- **4.7 Comando usado para corrigir problemas com blocos corrompidos, caso ocorra**:

```code
hdfs fsck / | egrep -v '^\.+$' | grep -v replica | grep -v Replica
```

<br>

- **4.8 Para deixar o modo de segurança do Hadoop caso ocorrar algum problema** (tem que estar logado como usário hadoop):

```code
hdfs dfsadmin -safemode leave
```


<br><br><br>

<br><br><br>




<br><br><br>

In [ ]:
sqlldr userid=curso/digitar_senha control=loader.dat log=loader.log

sqoop import --connect jdbc:oracle:thin:aluno/dsahadoop@dataserver.localdomain:1539/orcl --username aluno -password sua_senha --query "select user_id, movie_id from cinema where rating = 1 and \$CONDITIONS" --target-dir /user/oracle/output -m 1

sqoop import --connect jdbc:oracle:thin:curso/0804@dataserver.localdomain:1539/orcl --username curso --password 0804 --query "SELECT ID, NAME, DOMAIN, YEAR_FOUNDED, INDUSTRY, SIZE_RANGE, LOCALITY, COUNTRY, LINKEDIN_URL, CURRENT_EMPLOYEE_ESTIMATE, TOTAL_EMPLOYEE_ESTIMATE FROM COMPANIES WHERE \$CONDITIONS" --target-dir /user/oracle/output/companies_data --delete-target-dir --fields-terminated-by ',' --lines-terminated-by '\n' --null-string '\\N' --null-non-string '\\N' -m 1




sqoop import --connect jdbc:oracle:thin:aluno/dsahadoop@dataserver.localdomain:1539/orcl --username aluno -password sua_senha --query "select user_id, movie_id from cinema where rating = 1 and \$CONDITIONS" --target-dir /user/oracle/output -m 1


In [ ]:
sqoop eval --connect jdbc:oracle:thin:@dataserver.localdomain:1539/orcl --username curso --password 0804 --query "SELECT SYSDATE FROM DUAL;"
